In [1]:
from scipy.io import netcdf

import iris
import iris.coord_categorisation

import os
import logging

import numpy as np

import pickle
from pathlib import Path
logger = logging.getLogger(__name__)

In [ ]:
def get_scenarios(LSM):
    """
    Return a List containing SCENARIO tag names, not including the 
    no land use change scenarios. The land use scenario 
    names depend on the LSM. For now it will only return two climate
    scenarios (some models additionally ran 585). 

    Parameters
    ----------
    LSM : str
        Which land surface model is being run. Options are ``'clm'``,
        ``'jules'``, ``'orchidee'``, or ``'jsbach'``.

    Returns
    -------
    SCENARIO_TITLES: character
    PLOT_TITLES: character
    CLIM_SCENARIOS: character
    CLIM_MODELS: character      
    """
    CLIM_SCENARIOS = [ 'ssp126', 'ssp370' ]
    CLIM_MODELS = [ 'mpi-esm1-2-hr','ukesm1-0-ll','ipsl-cm6a-lr' ]

    if LSM=='clm':
        SCENARIO_TITLES = ['agtobio', 'agtonat', 'agtoaff', 'nattobio', 'nattoaff']
        PLOT_TITLES = ['Biocrops on ag land', 'Natural regrowth on ag land', 
                   'Afforestation on ag land', 'Biocrops on natural land',
                   'Afforestation on natural land']         
    elif LSM=='jules':
        SCENARIO_TITLES = ['agtobio', 'agtonat', 'agtoaff', 'nattobio', 'nattoaff']
        PLOT_TITLES = ['Biocrops on ag land', 'Natural regrowth on ag land', 
                   'Afforestation on ag land', 'Biocrops on natural land',
                   'Afforestation on natural land']
    elif LSM=='orchidee':
        SCENARIO_TITLES = ['agtoaff','nattoaff']
        PLOT_TITLES = ['Afforestation on ag land', 'Afforestation on natural land']         
    elif LSM=='jsbach':
        SCENARIO_TITLES = ['agtobio', 'agtoaff', 'nattobio']
        PLOT_TITLES = ['Biocrops on ag land',  
                   'Afforestation on ag land', 'Biocrops on natural land']        
    else:
        print('get_scenarios is not set up for this model:',LSM)
        
    return SCENARIO_TITLES, PLOT_TITLES, CLIM_SCENARIOS, CLIM_MODELS

In [2]:
def add_metadata(cube, field, fname):

    lsm_id = fname.split('/')[4]
    if lsm_id=='orchidee':
        esm_id = fname.split('_')[3]
        ssp_id = fname.split('_')[4]
    else:
        esm_id = fname.split('_')[1]
        ssp_id = fname.split('_')[2]

    iris.std_names.STD_NAMES['experiment_id'] = {'canonical units':'1'}
    
    #if not cube.coords("experiment_id"):
    climate_coord = iris.coords.AuxCoord(ssp_id, "experiment_id", units="1", long_name="experiment_id")
    cube.add_aux_coord(climate_coord)

    #if not cube.coords("realization"):
    esm_coord = iris.coords.AuxCoord(esm_id, "realization", units="1")
    cube.add_aux_coord(esm_coord)


In [3]:
def _get_land_fraction(cube):
    """Extract land fraction as :mod:`dask.array`."""
    fx_cube = None
    land_fraction = None
    errors = []
    
    try:
        fx_cube = cube.ancillary_variable('land_area_fraction')
    except iris.exceptions.AncillaryVariableNotFoundError:
        try:
            fx_cube = cube.ancillary_variable('sea_area_fraction')
        except iris.exceptions.AncillaryVariableNotFoundError:
            errors.append('Ancillary variables land/sea area fraction not '
                          'found in cube. Check ancillary data availability.')
            return (land_fraction, errors)

    land_fraction = fx_cube.core_data()
    
    if fx_cube.var_name == 'sftlf':
        land_fraction = fx_cube.core_data() / 100.0
    if fx_cube.var_name == 'sftof':
        land_fraction = 1.0 - fx_cube.core_data() / 100.0
    
    return (land_fraction, errors)

In [4]:
def weighting_landsea_fraction(cube, area_type):
    """Weight fields using land or sea fraction.

    This preprocessor function weights a field with its corresponding land or
    sea area fraction (value between 0 and 1). The application of this is
    important for most carbon cycle variables (and other land-surface outputs),
    which are e.g. reported in units of `kgC m-2`. This actually refers to 'per
    square meter of land/sea' and NOT 'per square meter of gridbox'. So in
    order to integrate these globally or regionally one has to both area-weight
    the quantity but also weight by the land/sea fraction.

    Parameters
    ----------
    cube : iris.cube.Cube
        Data cube to be weighted. It should have an
        :class:`iris.coords.AncillaryVariable` with standard name
        ``'land_area_fraction'`` or ``'sea_area_fraction'``. If both are
        present, only the ``'land_area_fraction'`` will be used.
    area_type : str
        Use land (``'land'``) or sea (``'sea'``) fraction for weighting.

    Returns
    -------
    iris.cube.Cube
        Land/sea fraction weighted cube.

    Raises
    ------
    TypeError
        ``area_type`` is not ``'land'`` or ``'sea'``.
    ValueError
        Land/sea fraction variables ``sftlf`` or ``sftof`` not found.
    """
    if area_type not in ('land', 'sea'):
        raise TypeError(
            f"Expected 'land' or 'sea' for area_type, got '{area_type}'")
    (land_fraction, errors) = _get_land_fraction(cube)
    if land_fraction is None:
        raise ValueError(
            f"Weighting of '{cube.var_name}' with '{area_type}' fraction "
            f"failed because of the following errors: {' '.join(errors)}")

    core_data = cube.core_data()
    if area_type == 'land':
        cube.data = core_data * land_fraction
    elif area_type == 'sea':
        cube.data = core_data * (1.0 - land_fraction)
    return cube

In [5]:
def add_ancillary_variable(cube, temp_ancil_cube):
    """Add cube as an ancillary variable in the cube containing the data.

    Parameters
    ----------
    cube: iris.cube.Cube
        Iris cube with input data.
    temp_ancil_cube: iris.cube.Cube
        Iris cube with ancillary data.

    Returns
    -------
    iris.cube.Cube
        Cube with added ancillary variables
    """
    
    cube_left_lat = cube.coord("longitude").points[0]
    cube_right_lat = cube.coord("longitude").points[-1]

    acube_left_lat = temp_ancil_cube.coord("longitude").points[0]
    acube_right_lat = temp_ancil_cube.coord("longitude").points[-1]

    if cube_left_lat != acube_left_lat:
        ancillary_cube = temp_ancil_cube.intersection(longitude=(-180,180), ignore_bounds=True )
    else:
        ancillary_cube = temp_ancil_cube
        
    ancillary_var = iris.coords.AncillaryVariable(
        ancillary_cube.core_data(),
        standard_name=ancillary_cube.standard_name,
        units=ancillary_cube.units,
        var_name=ancillary_cube.var_name,
        attributes=ancillary_cube.attributes)
    start_dim = cube.ndim - len(ancillary_var.shape)
    cube.add_ancillary_variable(ancillary_var, range(start_dim, cube.ndim))
    logger.debug('Added %s as ancillary variable in cube of %s.',
                 ancillary_cube.var_name, cube.var_name)

In [6]:
def retrieve_data_generic(var,LU_SCENARIO,LSM):
    """Get the outputs from the netcdf files and compute annual means.

    Parameters
    ----------
    var: character
        The variable to retrieve (cSoil and cVeg).
    LU_SCENARIO: character
        The land use scenario.
    LSM: character
        The land surface model (jules, clm, orchidee, jsbach)

    Returns
    -------
    ENSEMBLE_MEAN
        Dictionary with the global, annual mean outputs for plotting.
    """
    
    dum1,dum2,ssp, esm = get_scenarios(LSM)

    path = ''

    nclim = len(esm)
    years = np.arange(2015,2101)

    if LSM=='jules':
        land_frac = iris.load_cube('/home/aharper/mask_latlon2d.nc','land_sea_mask')
        land_frac.standard_name='land_area_fraction'
    elif LSM=='clm':
        land_frac = iris.load_cube('/home/aharper/clm_landfrac.nc','landfrac')
        land_frac.standard_name='land_area_fraction'
    elif LSM=='orchidee':
        land_frac = iris.load_cube('/home/aharper/orchidee_oceanCoverFrac_notime.nc','oceanCoverFrac')
        land_frac.standard_name='land_area_fraction' #the fact that its sea is accounted for below
        
    
    filenames = ['/bdd/ESM2025/WP10/'+LSM+'/'+LSM+'_mpi-esm1-2-hr_ssp126_'+LU_SCENARIO+'_'+var+'.monthly.nc',
                 '/bdd/ESM2025/WP10/'+LSM+'/'+LSM+'_ukesm1-0-ll_ssp126_'+LU_SCENARIO+'_'+var+'.monthly.nc',
                 '/bdd/ESM2025/WP10/'+LSM+'/'+LSM+'_ipsl-cm6a-lr_ssp126_'+LU_SCENARIO+'_'+var+'.monthly.nc',
                 '/bdd/ESM2025/WP10/'+LSM+'/'+LSM+'_mpi-esm1-2-hr_ssp370_'+LU_SCENARIO+'_'+var+'.monthly.nc',
                 '/bdd/ESM2025/WP10/'+LSM+'/'+LSM+'_ukesm1-0-ll_ssp370_'+LU_SCENARIO+'_'+var+'.monthly.nc',
                 '/bdd/ESM2025/WP10/'+LSM+'/'+LSM+'_ipsl-cm6a-lr_ssp370_'+LU_SCENARIO+'_'+var+'.monthly.nc']
    if LSM=='orchidee':
        filenames = ['/bdd/ESM2025/WP10/'+LSM+'/mpi-esm1-2-hr_ssp126_'+LU_SCENARIO+'/'+LSM+'_mpi-esm1-2-hr_ssp126_'+LU_SCENARIO+'_'+var+'.nc',
                 '/bdd/ESM2025/WP10/'+LSM+'/ukesm1-0-ll_ssp126_'+LU_SCENARIO+'/'+LSM+'_ukesm1-0-ll_ssp126_'+LU_SCENARIO+'_'+var+'.nc',
                 '/bdd/ESM2025/WP10/'+LSM+'/ipsl-cm6a-lr_ssp126_'+LU_SCENARIO+'/'+LSM+'_ipsl-cm6a-lr_ssp126_'+LU_SCENARIO+'_'+var+'.nc',
                 '/bdd/ESM2025/WP10/'+LSM+'/mpi-esm1-2-hr_ssp370_'+LU_SCENARIO+'/'+LSM+'_mpi-esm1-2-hr_ssp370_'+LU_SCENARIO+'_'+var+'.nc',
                 '/bdd/ESM2025/WP10/'+LSM+'/ukesm1-0-ll_ssp370_'+LU_SCENARIO+'/'+LSM+'_ukesm1-0-ll_ssp370_'+LU_SCENARIO+'_'+var+'.nc',
                 '/bdd/ESM2025/WP10/'+LSM+'/ipsl-cm6a-lr_ssp370_'+LU_SCENARIO+'/'+LSM+'_ipsl-cm6a-lr_ssp370_'+LU_SCENARIO+'_'+var+'.nc']
        
    my_file = Path('/bdd/ESM2025/WP10/analysis/files/'+LSM+'_'+var+'_'+LU_SCENARIO+'.pkl')
    tropics_file = Path('/bdd/ESM2025/WP10/analysis/files/'+LSM+'_'+var+'_'+LU_SCENARIO+'_tropics.pkl')
    temperate_file = Path('/bdd/ESM2025/WP10/analysis/files/'+LSM+'_'+var+'_'+LU_SCENARIO+'_temp.pkl')
    firstmap_file = Path('/bdd/ESM2025/WP10/analysis/files/'+LSM+'_'+var+'_'+LU_SCENARIO+'_2015.pkl')
    lastmap_file = Path('/bdd/ESM2025/WP10/analysis/files/'+LSM+'_'+var+'_'+LU_SCENARIO+'_2100.pkl')
    print(my_file)

    if temperate_file.is_file():
        # file exists, we can skip a bunch of steps:
        print('read pickle files')
        with open(my_file, 'rb') as f1:
            ENSEMBLE_MEAN = pickle.load(f1)
        with open(tropics_file, 'rb') as f2:
            TROPICS_MEAN = pickle.load(f2)
        with open(temperate_file, 'rb') as f3:    
            TEMPERATE_MEAN = pickle.load(f3)
  
    else:
        # file doesn't exist, loop through scenarios and get data
        esms = iris.load(filenames, callback=add_metadata)

        ENSEMBLE_MEAN = {ssp_scen: {clim_mod: [] for clim_mod in esm} for ssp_scen in ssp}
        TROPICS_MEAN = {ssp_scen: {clim_mod: [] for clim_mod in esm} for ssp_scen in ssp}
        TEMPERATE_MEAN = {ssp_scen: {clim_mod: [] for clim_mod in esm} for ssp_scen in ssp}
        FIRST_TIMESTEP = {ssp_scen: {clim_mod: [] for clim_mod in esm} for ssp_scen in ssp}
        LAST_TIMESTEP = {ssp_scen: {clim_mod: [] for clim_mod in esm} for ssp_scen in ssp}

        tropics_start=0
        NHland_start=0
        NHland_end=0
        if LSM=='jules':
            borderlat=23
        elif LSM=='clm':
            borderlat=25
        elif LSM=='orchidee':
            borderlat=23
        
        
        for ssp_id in ssp:
        
            for esm_id in esm:
                esm_cube = esms.extract_cube(iris.Constraint(realization=esm_id,experiment_id=ssp_id) )

                add_ancillary_variable(esm_cube,land_frac)
                
                latcoord = esm_cube.coord("latitude")
                latcoord.units="degrees"
                loncoord = esm_cube.coord("longitude")
                loncoord.units="degrees"        
    
                if tropics_start==0:
                    lats = latcoord.points
                    nlat = len(lats)
                    minlat=lats[0]
                    for i0 in range(nlat):
                        if lats[i0] > -25 and tropics_start==0:
                            tropics_start=i0
                            #print('beginning of tropics')
                        elif lats[i0] > borderlat and NHland_start==0:
                            tropics_end=i0
                            NHland_start=i0
                            #print('between tropics and temperate')
                        elif lats[i0] > 66 and NHland_end==0:
                            NHland_end=i0
                            #print('end of temperate zone')
                print('double check zones','Tropics:',lats[tropics_start],lats[tropics_end-1],\
                      'Temperate:',lats[NHland_start],lats[NHland_end-1])
                
                tcoord = esm_cube.coord("time")
                if LSM=='clm':
                    tcoord.units="days since 2015-01-01 0:0:0"
                elif LSM=='jules':
                    tcoord.units="seconds since 2015-01-01 0:0:0"
                else:
                    tcoord.units="days since 2015-01-01 0:0:0"
                
                tcoord.calendar="calendar_no_leap"
                firstdate = tcoord.units.num2date(tcoord.points[0])
                iris.coord_categorisation.add_year(esm_cube,'time',name='year')        
        
                if LSM=='jules' or LSM=='clm':
                    weighting_landsea_fraction(esm_cube, 'land')
                elif LSM=='orchidee':
                    weighting_landsea_fraction(esm_cube, 'sea')

                #Calculates the annual mean from monthly values.
                cveg_annual_map = esm_cube.aggregated_by('year',iris.analysis.MEAN)
                
                cveg_annual_map.coord("latitude").guess_bounds()
                cveg_annual_map.coord("longitude").guess_bounds() 

                #Previously the map routine was here.
                
                FIRST_TIMESTEP[ssp_id][esm_id].append(cveg_annual_map[0,].core_data())
                LAST_TIMESTEP[ssp_id][esm_id].append(cveg_annual_map[-1,].core_data())
                
                #Next calculate the global total
       
                grid_areas = iris.analysis.cartography.area_weights(cveg_annual_map)
                
                cveg_global = (cveg_annual_map.collapsed(["latitude","longitude"], iris.analysis.SUM, weights=grid_areas)/1.0e12)
                tropics = cveg_annual_map[:,tropics_start:tropics_end,:]
                NHland = cveg_annual_map[:,NHland_start:NHland_end,:]
                cveg_tropics = (tropics.collapsed(["latitude","longitude"], iris.analysis.SUM, weights=grid_areas[:,tropics_start:tropics_end,])/1.0e12)
                cveg_NHland = (NHland.collapsed(["latitude","longitude"], iris.analysis.SUM, weights=grid_areas[:,NHland_start:NHland_end,])/1.0e12)
                
                #The data is lazy we need to make it real:
                cveg_global.data
                cveg_tropics.data
                cveg_NHland.data
        
                #Calculate the mean across the 3 climate models for each SSP scenario.
                #ensemble_mean = cveg_global.collapsed("realization",iris.analysis.MEAN)
                ENSEMBLE_MEAN[ssp_id][esm_id].append(cveg_global.core_data())
                TROPICS_MEAN[ssp_id][esm_id].append(cveg_tropics.core_data())
                TEMPERATE_MEAN[ssp_id][esm_id].append(cveg_NHland.core_data())

        # If we've gone through this loop because the outputs files didn't exist:
        with open(my_file, 'wb') as f:
            pickle.dump(ENSEMBLE_MEAN, f)
        with open(tropics_file, 'wb') as f:
            pickle.dump(TROPICS_MEAN, f)            
        with open(temperate_file, 'wb') as f:
            pickle.dump(TEMPERATE_MEAN, f)        
        with open(firstmap_file, 'wb') as f2:
            pickle.dump(FIRST_TIMESTEP, f2)
        with open(lastmap_file, 'wb') as f3:
            pickle.dump(LAST_TIMESTEP, f3)

    return ENSEMBLE_MEAN,TROPICS_MEAN,TEMPERATE_MEAN
    